In [2]:
# ===============================
# CIFAR-10
# ===============================
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

tf.random.set_seed(42)
np.random.seed(42)

In [3]:
# ========================================
# 경로 설정
# ========================================
DATA_DIR = '/content/drive/MyDrive/Col/머신러닝 6주차'
os.makedirs(DATA_DIR, exist_ok=True)

In [4]:
# --- CIFAR-10 로드 (keras는 기본적으로 ~/.keras/datasets 에 캐시) ---
cifar10_path = os.path.join(DATA_DIR, 'cifar-10-batches-py')
if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터(사용자 폴더) 발견: {cifar10_path}")
else:
    print(f"데이터가 없습니다. (keras 기본 경로로) 다운로드/캐싱합니다...")

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print("✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}, 라벨: {y_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}, 라벨: {y_test.shape}\n")

데이터가 없습니다. (keras 기본 경로로) 다운로드/캐싱합니다...
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3), 라벨: (50000, 1)
  - 테스트 데이터: (10000, 32, 32, 3), 라벨: (10000, 1)



In [5]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [6]:
# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c) for c in selected_classes]

In [7]:
# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask  = np.isin(y_test,  selected_idx).flatten()

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test,  y_test  = x_test[test_mask],  y_test[test_mask]

print(f"선택된 클래스: {selected_classes} -> 인덱스 {selected_idx}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

선택된 클래스: ['cat', 'dog', 'horse'] -> 인덱스 [3, 5, 7]
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [8]:
# --- 라벨을 0~2로 재매핑 ---
label_map = {v: i for i, v in enumerate(selected_idx)}   # 예: {3:0, 5:1, 7:2}
y_train = np.array([label_map[int(y)] for y in y_train.flatten()], dtype=np.int64)
y_test  = np.array([label_map[int(y)] for y in y_test.flatten()],  dtype=np.int64)

In [9]:
# --- 정규화 ---
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

print("라벨 매핑 확인:")
print("  - train 라벨 고유값:", np.unique(y_train))
print("  - test  라벨 고유값:", np.unique(y_test))
print(f"최종 텐서 크기: x_train {x_train.shape}, y_train {y_train.shape}")
print(f"               x_test  {x_test.shape},  y_test  {y_test.shape}\n")

라벨 매핑 확인:
  - train 라벨 고유값: [0 1 2]
  - test  라벨 고유값: [0 1 2]
최종 텐서 크기: x_train (15000, 32, 32, 3), y_train (15000,)
               x_test  (3000, 32, 32, 3),  y_test  (3000,)



In [10]:
# --- 학습/검증 분할 ---
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
print(f"학습/검증 분할 -> train: {x_train.shape[0]}, val: {x_val.shape[0]}")

학습/검증 분할 -> train: 12000, val: 3000


In [11]:
# --- 데이터 증강 레이어 ---
data_augment = tf.keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
    ],
    name="data_augment"
)

In [12]:
# --- 모델 정의 ---
def build_model(input_shape=(32,32,3), num_classes=3):
    inputs = layers.Input(shape=input_shape)
    x = data_augment(inputs)

    # Conv Block 1
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Conv Block 2
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Conv Block 3
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name="cifar10_3class_cnn")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()

Model: "cifar10_3class_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augment (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 812,323 (3.10 MB)

 Trainable params: 812,323 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# --- 콜백 설정 ---
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=8, restore_best_weights=True, monitor='val_accuracy'
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5, patience=3, verbose=1, min_lr=1e-6, monitor='val_loss'
)
checkpoint_path = os.path.join(DATA_DIR, 'best_cifar10_3class.keras')
ckpt = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max'
)

In [14]:
# --- 학습 ---
BATCH_SIZE = 128
EPOCHS = 40

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, reduce_lr, ckpt],
    verbose=1,
)

Epoch 1/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.4130 - loss: 1.0522 - val_accuracy: 0.5657 - val_loss: 0.8718 - learning_rate: 0.0010
Epoch 2/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5426 - loss: 0.9155 - val_accuracy: 0.5927 - val_loss: 0.8307 - learning_rate: 0.0010
Epoch 3/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6002 - loss: 0.8335 - val_accuracy: 0.6197 - val_loss: 0.7919 - learning_rate: 0.0010
Epoch 4/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6171 - loss: 0.7960 - val_accuracy: 0.6583 - val_loss: 0.7467 - learning_rate: 0.0010
Epoch 5/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6488 - loss: 0.7552 - val_accuracy: 0.6787 - val_loss: 0.7123 - learning_rate: 0.0010
Epoch 6/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6572 - loss: 0.7415 - val_accuracy: 0.6823 - val_loss: 0.6816 - learning_rate: 0.0010
Epoch 7/40
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6755 - loss: 0.7128 - val_acc

In [15]:
# --- 평가 ---
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n[테스트 성능] loss={test_loss:.4f}, acc={test_acc:.4f}")


[테스트 성능] loss=0.4389, acc=0.8297
